In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
# Importing the glass.data file into a pandas dataframe
glassdf = pd.read_csv('glass.data')
glassdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   214 non-null    int64  
 1   RI      214 non-null    float64
 2   Na      214 non-null    float64
 3   Mg      214 non-null    float64
 4   Al      214 non-null    float64
 5   Si      214 non-null    float64
 6   K       214 non-null    float64
 7   Ca      214 non-null    float64
 8   Ba      214 non-null    float64
 9   Fe      214 non-null    float64
 10  Class   214 non-null    int64  
dtypes: float64(9), int64(2)
memory usage: 18.5 KB


In [3]:
glassdf.head()

,index,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [4]:
glassdf.dtypes

index      int64
RI       float64
Na       float64
Mg       float64
Al       float64
Si       float64
K        float64
Ca       float64
Ba       float64
Fe       float64
Class      int64
dtype: object

In [5]:
try:
    conn = mysql.connector.connect(host='localhost', user='root', password='12345') 
                        
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("DROP DATABASE IF EXISTS glassdb")
        print('Existing database glassdb dropped')
        cursor.execute("CREATE DATABASE glassdb")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Existing database glassdb dropped
Database is created


In [6]:
try:
#conn = mysql.connect(host='localhost', database='employee', user='root', password='root@123') #THis line of code is not required since we have already connected to db
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("use glassdb;")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS glass;')
        print('Creating table....')
# Note:- Faced an error in while creating the table since the the first column name in the dataset was index which is a keyword in mysql.
# changed the column name to Index_ and code executed fine
        cursor.execute("""CREATE TABLE glass(Index_ int, RI float, Na float, Mg float, Al float, Si float, K  float, Ca float, Ba float, Fe float, Class int)""")
        print("Table is created....")
        #loop through the data frame
        count=0
        for i,row in glassdf.iterrows():
            #here %S means string values 
            sql = "INSERT INTO glassdb.glass VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
#            print("Record inserted")
            count=count+1
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
        print(f'{count} records inserted')
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  None
Creating table....
Table is created....
214 records inserted


In [7]:
# Execute query
sql = "SELECT * FROM glassdb.glass"
cursor.execute(sql)
# Fetch all the records
result = cursor.fetchall()
print('No. of records in a the table glass:', len(result))

No. of records in a the table glass: 214


In [12]:
# The iterrows() method generates an iterator object of the DataFrame, allowing us to iterate each row in the DataFrame.
# Each iteration produces an index object and a row object (a Pandas Series object).
# This how we are fetching each row from the dataset(dataframe) and inserting it to the sql table

#for index,row in glassdf.iterrows():
#    print(index,tuple(row))